In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import datetime
from scipy.sparse import csr_matrix
import joblib
import matplotlib.pyplot as plt
import scipy

In [3]:
MNM_nb_folder = os.path.join('/home/lemma/Documents/MAC-POSTS/side_project/network_builder')
sys.path.append(MNM_nb_folder)

In [4]:
from MNM_mcnb import *

In [5]:
data_folder = os.path.join('..' , '..', 'data', 'input_files_MckeesRocks_SPC_AM')

In [6]:
nb = MNM_network_builder()
nb.load_from_folder(data_folder)

In [7]:
nb.demand

MNM_demand, number of O: 290

In [8]:
nb.graph

MNM_graph, number of node:6317, number of edges:16144

In [9]:
# new_folder = "tmp_truck"

In [10]:
# nb.dump_to_folder(new_folder)

### Build other direction

In [11]:
other_direction_dict = dict()
for edge in nb.graph.G.edges_iter(data = True):
    edge_id = edge[2]['ID']
    from_node = edge[0]
    to_node = edge[1]
    if to_node in nb.graph.G and from_node in nb.graph.G[to_node]:
        other_edge_id = nb.graph.G[to_node][from_node]['ID']
        other_direction_dict[other_edge_id] = edge_id
        other_direction_dict[edge_id] = other_edge_id
#     break

## Time list

In [12]:
start_date = datetime.date(2014, 1, 1)
end_date = datetime.date(2014, 1, 1)
delta_date = datetime.timedelta(minutes = 15)
time_basis1 = datetime.time(5,0,0)
time_basis2 = datetime.time(11,59,0)
cur_date_time = datetime.datetime.combine(start_date, time_basis1)
end_date_time = datetime.datetime.combine(end_date, time_basis2)

date_need_to_finish = list()

while(cur_date_time <= end_date_time):
    no = cur_date_time.weekday()
    if no < 8:
        single_date = cur_date_time.time()
        date_need_to_finish.append(single_date)
    cur_date_time = cur_date_time + delta_date

In [13]:
len(date_need_to_finish)

28

## rebuild dataset

In [14]:
data_folder = os.path.join(".")
AM_str = "AM 7-12AM"
count_str = "final count"
spd_str = "final speed"

In [15]:
count_wo = pd.read_csv(os.path.join(data_folder, count_str, AM_str, 
                        'TMS without veh classification', 'smoothedCounts_7-12AM_6hours_noclass.csv'), 
                       header = None, index_col = 0)
count_car = pd.read_csv(os.path.join(data_folder, count_str, AM_str, 
                        'TMS with veh classification', 'smoothedCounts_7-12AM_6hours_car.csv'), 
                       header = None, index_col = 0)
count_truck = pd.read_csv(os.path.join(data_folder, count_str, AM_str, 
                        'TMS with veh classification', 'smoothedCounts_7-12AM_6hours_truck.csv'), 
                       header = None, index_col = 0)

In [16]:
count_wo.columns = ['Direction'] + date_need_to_finish
count_car.columns = ['Direction'] + date_need_to_finish
count_truck.columns = ['Direction'] + date_need_to_finish

In [17]:
def get_bi_link(df):
    df['OD'] = None
    link_list = df.index.tolist()
    for i in range(len(df)):
        linkID = link_list[i]
        if df.iat[i, 0] == 'Bidirection':
            if linkID in other_direction_dict.keys():
                other_link = other_direction_dict[linkID]
    #             print linkID, other_link
                df.iat[i, -1] = (linkID, other_link)
            else:
                print "Error", linkID
                df.iat[i, -1] = (linkID,)
        else:
            df.iat[i, -1] = (linkID,)
    return df

In [18]:
count_wo = get_bi_link(count_wo)
count_car = get_bi_link(count_car)
count_truck = get_bi_link(count_truck)

Error 2075
Error 2096
Error 2122
Error 2132
Error 2138
Error 10362
Error 2179
Error 10428
Error 4988
Error 2357
Error 4843
Error 6552
Error 16802
Error 12709
Error 10690
Error 8646
Error 10696
Error 10743
Error 4620
Error 2141
Error 16957
Error 19064
Error 12981
Error 21213
Error 19923
Error 13047
Error 17205
Error 7014
Error 9085
Error 10126
Error 11246
Error 9201
Error 14014
Error 21536
Error 17476
Error 11349
Error 15623
Error 13620
Error 5436
Error 13649
Error 9557
Error 9589
Error 15392
Error 17938
Error 16303
Error 17959
Error 20057
Error 14011
Error 1005251
Error 14031
Error 14060
Error 16836
Error 12065
Error 3914
Error 3928
Error 3938
Error 1913
Error 12066
Error 16305
Error 8160
Error 14312
Error 3930
Error 13499
Error 16305
Error 11663
Error 16811
Error 16816
Error 15843
Error 10817
Error 19014
Error 19019
Error 19042
Error 15297
Error 5036
Error 20201
Error 19923
Error 1969
Error 6970
Error 17236
Error 4988
Error 9095
Error 17305
Error 4068
Error 20462
Error 2075
Error 2096

In [19]:
spd_car = pd.read_csv(os.path.join(data_folder, spd_str, AM_str, 
                        'speed_7-12AM_6hours_car.csv'), 
                       header = None, index_col = 0)
spd_truck = pd.read_csv(os.path.join(data_folder, spd_str, AM_str, 
                        'speed_7-12AM_6hours_truck.csv'), 
                       header = None, index_col = 0)

In [20]:
spd_car.columns = date_need_to_finish
spd_truck.columns = date_need_to_finish

In [21]:
count_wo.head()

,Direction,05:00:00,05:15:00,05:30:00,05:45:00,06:00:00,06:15:00,06:30:00,06:45:00,07:00:00,...,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,11:30:00,11:45:00,OD
0,,,,,,,,,,,,,,,,,,,,,
1019904,Bidirection,11.02500,13.300,13.78125,14.2625,15.7500,19.00,19.6875,20.375,21.0625,...,22.250,24.1250,26.00,25.3750,24.750,24.1250,23.50,23.1250,22.750,"(1019904, 19895)"
1003522,Bidirection,67.33125,78.925,89.29375,99.6625,96.1875,112.75,127.5625,142.375,157.1875,...,91.875,84.9375,78.00,79.5625,81.125,82.6875,84.25,85.1250,86.000,"(1003522, 3522)"
1019910,Bidirection,9.05625,10.500,10.41250,10.3250,12.9375,15.00,14.8750,14.750,14.6250,...,14.625,14.5625,14.50,13.9375,13.375,12.8125,12.25,12.8750,13.500,"(1019910, 19894)"
6152,Single,122.58750,145.425,157.98125,170.5375,175.1250,207.75,225.6875,243.625,261.5625,...,220.000,216.5000,213.00,216.9375,220.875,224.8125,228.75,238.0000,247.250,"(6152,)"
6153,Single,134.96875,155.750,170.71250,185.6750,192.8125,222.50,243.8750,265.250,286.6250,...,237.625,231.1875,224.75,224.6250,224.500,224.3750,224.25,227.1875,230.125,"(6153,)"


In [22]:
count_car.head()

,Direction,05:00:00,05:15:00,05:30:00,05:45:00,06:00:00,06:15:00,06:30:00,06:45:00,07:00:00,...,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,11:30:00,11:45:00,OD
0,,,,,,,,,,,,,,,,,,,,,
14337,Single,344.48750,398.475,408.45000,418.4250,492.1250,569.25,583.5000,597.750,612.0000,...,399.500,401.0000,402.50,402.6250,402.750,402.8750,403.00,414.5625,426.125,"(14337,)"
16386,Single,499.45000,599.200,617.83750,636.4750,713.5000,856.00,882.6250,909.250,935.8750,...,538.250,499.7500,461.25,456.3750,451.500,446.6250,441.75,436.4375,431.125,"(16386,)"
4101,Bidirection,41.12500,47.950,62.82500,77.7000,58.7500,68.50,89.7500,111.000,132.2500,...,92.375,90.1875,88.00,90.3125,92.625,94.9375,97.25,99.1875,101.125,"(4101, 1004101)"
1003526,Bidirection,60.76875,71.400,77.83125,84.2625,86.8125,102.00,111.1875,120.375,129.5625,...,89.000,86.5000,84.00,85.0625,86.125,87.1875,88.25,90.1250,92.000,"(1003526, 3526)"
1006593,Bidirection,95.06875,111.650,133.56875,155.4875,135.8125,159.50,190.8125,222.125,253.4375,...,299.125,303.3125,307.50,320.9375,334.375,347.8125,361.25,369.1875,377.125,"(1006593, 6593)"


In [23]:
count_truck.head()

,Direction,05:00:00,05:15:00,05:30:00,05:45:00,06:00:00,06:15:00,06:30:00,06:45:00,07:00:00,...,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,11:30:00,11:45:00,OD
0,,,,,,,,,,,,,,,,,,,,,
14337,Single,40.15625,43.750,43.43750,43.1250,80.3125,87.50,86.8750,86.250,85.6250,...,82.875,85.9375,89.00,87.5625,86.125,84.6875,83.25,81.1250,79.000,"(14337,)"
16386,Single,22.56250,26.000,23.93750,21.8750,45.1250,52.00,47.8750,43.750,39.6250,...,46.125,45.4375,44.75,44.1875,43.625,43.0625,42.50,39.8750,37.250,"(16386,)"
4101,Bidirection,2.65625,3.000,3.34375,3.6875,5.3125,6.00,6.6875,7.375,8.0625,...,7.125,6.8125,6.50,6.5625,6.625,6.6875,6.75,6.3750,6.000,"(4101, 1004101)"
1003526,Bidirection,1.50000,1.875,2.00000,2.1250,3.0000,3.75,4.0000,4.250,4.5000,...,3.750,3.7500,3.75,4.0625,4.375,4.6875,5.00,5.0000,5.000,"(1003526, 3526)"
1006593,Bidirection,5.18750,5.375,6.28125,7.1875,10.3750,10.75,12.5625,14.375,16.1875,...,21.500,20.6250,19.75,20.3125,20.875,21.4375,22.00,22.6875,23.375,"(1006593, 6593)"


## constructing

In [24]:
nb_link_ID_list = pickle.load(open('../path_buffer_adding/nb_link_ID_list.pickle', 'rb'))

In [25]:
count_car = count_car[~count_car.index.duplicated()]
count_truck = count_truck[~count_truck.index.duplicated()]
spd_truck = spd_truck[~spd_truck.index.duplicated()]
spd_car = spd_car[~spd_car.index.duplicated()]

In [26]:
class_count_list = [count_car, count_truck]
class_spd_list = [spd_car, spd_truck]

In [27]:
observed_link_list = list()
for i in range(len(count_wo)):
    ids = count_wo.iat[i, -1]
    if len(ids) == 1:
        observed_link_list.append(ids[0])
    else:
        observed_link_list.append(ids[0])
        observed_link_list.append(ids[1])
for k in range(len(class_count_list)):
    for i in range(len(class_count_list[k])):
        ids = class_count_list[k].iat[i, -1]
        if len(ids) == 1:
            observed_link_list.append(ids[0])
        else:
            observed_link_list.append(ids[0])
            observed_link_list.append(ids[1])
observed_link_list = sorted(list(set(observed_link_list + spd_car.index.tolist() + spd_truck.index.tolist())))
print len(observed_link_list)
observed_link_list = list(filter(lambda x: x in nb_link_ID_list, observed_link_list))
print len(observed_link_list)

2918
2916


In [28]:
len(set(observed_link_list))

2916

In [29]:
count_car = count_car.reindex(observed_link_list)
count_truck = count_truck.reindex(observed_link_list)
spd_truck = spd_truck.reindex(observed_link_list)
spd_car = spd_car.reindex(observed_link_list)

In [30]:
def construct_mL(mat, start_interval, number_of_intervales, link_list, observed_link_list):
    num_total_links = len(observed_link_list)
    m = mat[:, start_interval : start_interval + number_of_intervales].flatten(order = 'F')
    R = mat.shape[0]
    T = number_of_intervales
    x_list = list()
    y_list = list()
    data_list = list()
    for i in range(R):
        link_ids = link_list[i]
        try:
            link_inds = list(map(lambda x: observed_link_list.index(x), link_ids))
            for j in range(T):
                for link_ind in link_inds:
                    x = j * num_total_links + link_ind
                    y = j * R + i
                    data = 1
                    x_list.append(x)
                    y_list.append(y)
                    data_list.append(data)
        except:
            continue
    L = csr_matrix((data_list, (y_list, x_list)), shape=(R * T, num_total_links * T))
    return m, L

In [31]:
def construct_mL2(mat, start_interval, number_of_intervales, link_list, observed_link_list):
    num_total_links = len(observed_link_list)
    m = mat[:, start_interval : start_interval + number_of_intervales].flatten(order = 'F')
    R = mat.shape[0]
    T = number_of_intervales
    x_list = list()
    y_list = list()
    data_list = list()
    for i in range(R):
        link_ids = link_list[i]
        try:
            link_inds = list(map(lambda x: observed_link_list.index(x), link_ids))
            for link_ind in link_inds:
                x = link_ind
                y = i
                data = 1
                x_list.append(x)
                y_list.append(y)
                data_list.append(data)
        except:
            continue
    L_one = csr_matrix((data_list, (y_list, x_list)), shape=(R, num_total_links))
    L = scipy.sparse.block_diag([L_one for t in range(T)])
    return m, L

In [32]:
# J_list = list()
# for k in range(len(class_count_list)):
#     m, L = construct_mL(class_count_list[k].loc[:, date_need_to_finish].values, 
#              len(date_need_to_finish), 
#              class_count_list[k]['OD'].tolist(), observed_link_list)
#     J_list.append((m, L))

In [33]:
m_car, L_car = construct_mL(count_car.loc[:, date_need_to_finish].values, 
             0, 28, 
             count_car['OD'].tolist(), observed_link_list)

In [34]:
m_car2, L_car2 = construct_mL2(count_car.loc[:, date_need_to_finish].values, 
             0, 28, 
             count_car['OD'].tolist(), observed_link_list)

In [35]:
np.abs((L_car2 - L_car)).sum()

0

In [36]:
m_truck, L_truck = construct_mL(count_truck.loc[:, date_need_to_finish].values, 
             0, 28, 
             count_truck['OD'].tolist(), observed_link_list)

In [37]:
m_spd_car = spd_car.loc[:, date_need_to_finish].values.flatten(order = 'F')
m_spd_truck = spd_truck.loc[:, date_need_to_finish].values.flatten(order = 'F')

In [38]:
# pickle.dump([m_car, L_car, m_truck, L_truck, observed_link_list], open('mc_count.pickle', 'wb'))

In [39]:
pickle.dump([m_car, L_car, m_truck, L_truck, m_spd_car, m_spd_truck, observed_link_list], open('mc_AM_data.pickle', 'wb'))

In [113]:
m_truck.shape

(69840,)

In [112]:
m_spd_car.shape

(69840,)

In [49]:
len(observed_link_list)

2209

## Smaller cared set

In [24]:
count_car = count_car[~count_car.index.duplicated()]
count_truck = count_truck[~count_truck.index.duplicated()]
spd_truck = spd_truck[~spd_truck.index.duplicated()]
spd_car = spd_car[~spd_car.index.duplicated()]

In [25]:
study_area = [99009, 1099009, 12056, 1012056, 12046, 12178, 1099012, 99012, 1005251, 1005234, 1005253]

In [26]:
nb_link_ID_list = pickle.load(open('../path_buffer_adding/nb_link_ID_list.pickle', 'rb'))

In [27]:
count_car = count_car[count_car.index.isin(study_area)]
count_truck = count_truck[count_truck.index.isin(study_area)]
spd_truck = spd_truck[spd_truck.index.isin(study_area)]
spd_car = spd_car[spd_car.index.isin(study_area)]

In [28]:
class_count_list = [count_car, count_truck]
class_spd_list = [spd_car, spd_truck]

In [29]:
observed_link_list = list()
# for i in range(len(count_wo)):
#     ids = count_wo.iat[i, -1]
#     if len(ids) == 1:
#         observed_link_list.append(ids[0])
#     else:
#         observed_link_list.append(ids[0])
#         observed_link_list.append(ids[1])
for k in range(len(class_count_list)):
    for i in range(len(class_count_list[k])):
        ids = class_count_list[k].iat[i, -1]
        if len(ids) == 1:
            observed_link_list.append(ids[0])
        else:
            observed_link_list.append(ids[0])
            observed_link_list.append(ids[1])
observed_link_list = sorted(list(set(observed_link_list + spd_car.index.tolist() + spd_truck.index.tolist())))
print len(observed_link_list)
observed_link_list = list(filter(lambda x: x in nb_link_ID_list, observed_link_list))
print len(observed_link_list)

12
11


In [30]:
observed_link_list

[5260,
 12046,
 12056,
 12178,
 99009,
 99012,
 1005234,
 1005253,
 1012056,
 1099009,
 1099012]

In [31]:
count_car = count_car.reindex(observed_link_list)
count_truck = count_truck.reindex(observed_link_list)
spd_truck = spd_truck.reindex(observed_link_list)
spd_car = spd_car.reindex(observed_link_list)

In [32]:
count_car

,Direction,05:00:00,05:15:00,05:30:00,05:45:00,06:00:00,06:15:00,06:30:00,06:45:00,07:00:00,...,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,11:30:00,11:45:00,OD
0,,,,,,,,,,,,,,,,,,,,,
5260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12046,Single,211.80250,217.9450,217.9450,217.9450,302.5750,311.350,311.350,311.350,302.575,...,299.000,327.9250,327.925,327.9250,327.925,336.7000,336.700,336.7000,336.700,"(12046,)"
12056,Single,178.36000,226.5900,226.5900,226.5900,254.8000,323.700,323.700,323.700,254.800,...,215.150,212.8750,212.875,212.8750,212.875,240.5000,240.500,240.5000,240.500,"(12056,)"
12178,Single,142.41500,144.6900,144.6900,144.6900,203.4500,206.700,206.700,206.700,203.450,...,221.325,238.5500,238.550,238.5500,238.550,247.9750,247.975,247.9750,247.975,"(12178,)"
99009,Single,183.59250,249.3400,249.3400,249.3400,262.2750,356.200,356.200,356.200,262.275,...,194.350,189.4750,189.475,189.4750,189.475,198.9000,198.900,198.9000,198.900,"(99009,)"
99012,Single,67.79500,77.8050,77.8050,77.8050,96.8500,111.150,111.150,111.150,96.850,...,92.625,87.1000,87.100,87.1000,87.100,92.3000,92.300,92.3000,92.300,"(99012,)"
1005234,Single,73.36875,85.4000,96.5125,107.6250,104.8125,122.000,137.875,153.750,169.625,...,156.625,161.8125,167.000,168.8125,170.625,172.4375,174.250,176.9375,179.625,"(1005234,)"
1005253,Bidirection,1.61875,1.9250,2.1000,2.2750,2.3125,2.750,3.000,3.250,3.500,...,8.500,8.3750,8.250,8.3750,8.500,8.6250,8.750,8.7500,8.750,"(1005253, 5260)"
1012056,Single,202.24750,214.5325,214.5325,214.5325,288.9250,306.475,306.475,306.475,288.925,...,260.325,278.2000,278.200,278.2000,278.200,294.4500,294.450,294.4500,294.450,"(1012056,)"


In [33]:
def construct_mL(mat, start_interval, number_of_intervales, link_list, observed_link_list):
    num_total_links = len(observed_link_list)
    m = mat[:, start_interval : start_interval + number_of_intervales].flatten(order = 'F')
    R = mat.shape[0]
    T = number_of_intervales
    x_list = list()
    y_list = list()
    data_list = list()
    for i in range(R):
        link_ids = link_list[i]
        try:
            link_inds = list(map(lambda x: observed_link_list.index(x), link_ids))
            for j in range(T):
                for link_ind in link_inds:
                    x = j * num_total_links + link_ind
                    y = j * R + i
                    data = 1
                    x_list.append(x)
                    y_list.append(y)
                    data_list.append(data)
        except:
            continue
    L = csr_matrix((data_list, (y_list, x_list)), shape=(R * T, num_total_links * T))
    return m, L

In [34]:
def construct_mL2(mat, start_interval, number_of_intervales, link_list, observed_link_list):
    num_total_links = len(observed_link_list)
    m = mat[:, start_interval : start_interval + number_of_intervales].flatten(order = 'F')
    R = mat.shape[0]
    T = number_of_intervales
    x_list = list()
    y_list = list()
    data_list = list()
    for i in range(R):
        link_ids = link_list[i]
        try:
            link_inds = list(map(lambda x: observed_link_list.index(x), link_ids))
            for link_ind in link_inds:
                x = link_ind
                y = i
                data = 1
                x_list.append(x)
                y_list.append(y)
                data_list.append(data)
        except:
            continue
    L_one = csr_matrix((data_list, (y_list, x_list)), shape=(R, num_total_links))
    L = scipy.sparse.block_diag([L_one for t in range(T)])
    return m, L

In [35]:
m_car, L_car = construct_mL(count_car.loc[:, date_need_to_finish].values, 
             0, 28, 
             count_car['OD'].tolist(), observed_link_list)

In [36]:
m_car2, L_car2 = construct_mL2(count_car.loc[:, date_need_to_finish].values, 
             0, 28, 
             count_car['OD'].tolist(), observed_link_list)

In [37]:
np.abs((L_car2 - L_car)).sum()

0

In [38]:
m_truck, L_truck = construct_mL(count_truck.loc[:, date_need_to_finish].values, 
             0, 28, 
             count_truck['OD'].tolist(), observed_link_list)

In [39]:
m_spd_car = spd_car.loc[:, date_need_to_finish].values.flatten(order = 'F')
m_spd_truck = spd_truck.loc[:, date_need_to_finish].values.flatten(order = 'F')

In [38]:
# pickle.dump([m_car, L_car, m_truck, L_truck, observed_link_list], open('mc_count.pickle', 'wb'))

In [40]:
pickle.dump([m_car, L_car, m_truck, L_truck, m_spd_car, m_spd_truck, observed_link_list], open('small_mc_AM_data.pickle', 'wb'))